# Text classification with movie reivews
<hr>

이 notebook은 영화리뷰의 텍스트를 이용하여 긍정인지 부정인지를 분류한다. 이러한 긍, 부정 분류는 2중 분류 (binary, or two-class) 한 예시이고, 머신러닝 문제에서 넓게 적용할 수 있다.

사용될 데이터셋인 [IMDB dataset](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb)는:
- 50,000 영화 리뷰 포함하며
- training과 testing set들은 같은 각 set 내에서 같은 수의 긍정, 부정 리뷰를 담고있다.

여기서는 model을 구성하고 학습하기 하기 위한 high-level API인 `tf.keras`를 사용하며, `tf.keras`를 사용하여 text classification에 대해 더 심화된 가이드를 얻고 싶다면 [MLCC Text Classification Guide](https://developers.google.com/machine-learning/guides/text-classification/) 여기를 참고하라.

In [7]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

1.5.0


## Download the IMDB dataset
<hr>

In [ ]:
IMDB dataset은 tensorflow로 다운받을 수 있다.

## Explore the data
<hr>

### convert the integers back to words

## Prepare the data
<hr>

## Build the model
<hr>

### Hidden units

### Loss function and optimizer

## Create a validation set
<hr>

## Train the model
<hr>

## Evaluate the model
<hr>

## Create a graph of accuracy and loss over time
<hr>